**Even though it's mentioned on huggingface that ced-base supports variable length audios for fine tuning it doesn't!**

In [1]:
from ced_model.feature_extraction_ced import CedFeatureExtractor
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Audio, concatenate_datasets, Dataset
import numpy as np
import torch
import json
import evaluate

In [2]:
with open("../config.json", mode = "r") as f:
    data = json.load(f)
    SAMPLING_RATE = data["sampling_rate"]
    SEGMENT_LEN = data["segment_length"]
    OVERLAP_LEN = data["overlap_length"]

In [3]:
extractor = CedFeatureExtractor.from_pretrained("mispeech/ced-base")

/users/labnet5/gr5/abahari/Documents/Fridays/BirdCallLabeling/env-labeling/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("Saads/xecanto_birds", split = "train")

In [5]:
dataset = dataset.remove_columns([
    "primary_label",
    "secondary_labels",
    "scientific_name",
    "author",
    "license",
    "rating",
    "type",
    "latitude",
    "longitude",
    "url"
])
dataset

Dataset({
    features: ['audio', 'common_name'],
    num_rows: 11031
})

In [6]:
dataset[0:3]

{'audio': [{'path': '/users/labnet5/gr5/abahari/.cache/huggingface/hub/datasets--Saads--xecanto_birds/snapshots/a0cca0425468f94c84acd14479327b43c3c06084/abethr1/XC128013.ogg',
   'array': array([-6.02842738e-06,  1.55201872e-06, -9.41774886e-07, ...,
          -1.04878731e-04,  9.59840952e-04,  9.13573662e-04]),
   'sampling_rate': 32000},
  {'path': '/users/labnet5/gr5/abahari/.cache/huggingface/hub/datasets--Saads--xecanto_birds/snapshots/a0cca0425468f94c84acd14479327b43c3c06084/abethr1/XC363501.ogg',
   'array': array([ 1.26495331e-06,  2.67615587e-06,  1.60536308e-06, ...,
          -2.96967191e-05,  5.09194106e-05,  1.82838030e-05]),
   'sampling_rate': 32000},
  {'path': '/users/labnet5/gr5/abahari/.cache/huggingface/hub/datasets--Saads--xecanto_birds/snapshots/a0cca0425468f94c84acd14479327b43c3c06084/abethr1/XC363502.ogg',
   'array': array([ 4.54875226e-06,  3.57464123e-05, -4.50542643e-07, ...,
           5.83382962e-06, -1.26050472e-05,  2.66156712e-06]),
   'sampling_rate': 

In [7]:
def count_chunks(audio_array, chunk_length = SEGMENT_LEN, overlap = OVERLAP_LEN):
    chunk_length = chunk_length * SAMPLING_RATE
    overlap = overlap * SAMPLING_RATE

    counter = 0
    start = 0
    while start + chunk_length <= len(audio_array):
        counter += 1
        start += (chunk_length - overlap)
    
    if start < len(audio_array):
        counter += 1
    
    return counter

In [8]:
def get_count_chunks(row):
    counter = count_chunks(row["audio"]["array"])
    return {"num_audio_chunks": counter}

In [9]:
dataset = dataset.cast_column("audio", Audio(sampling_rate = SAMPLING_RATE))
dataset_chunked = dataset.map(
    get_count_chunks,
    remove_columns = "audio",
    batched = False,
    num_proc = 16,
    writer_batch_size = 1000
)

Map (num_proc=16):   0%|          | 0/11031 [00:00<?, ? examples/s]

In [10]:
df_chunked = dataset_chunked.to_pandas()
df_chunked = df_chunked.groupby(
    "common_name",
    as_index = False
).sum().sort_values(
    by = "num_audio_chunks",
    ascending = False
).reset_index(
    drop = True
)
df_chunked

,common_name,num_audio_chunks
0,Thrush Nightingale,1187
1,Lawrence's Goldfinch,926
2,Common Buzzard,894
3,Common House-Martin,881
4,Willow Warbler,871
...,...,...
259,Red-headed Bluebill,3
260,Goliath Heron,2
261,White-headed Sawwing,2
262,White-crested Turaco,1


In [11]:
selected_labels = df_chunked[df_chunked["num_audio_chunks"] >= 350]["common_name"].to_list()
len(selected_labels)

54

In [12]:
dataset = dataset.filter(
    lambda row: row["common_name"] in selected_labels,
    batched = False,
    num_proc = 4,
    writer_batch_size = 50
)

Filter (num_proc=4):   0%|          | 0/11031 [00:00<?, ? examples/s]

In [13]:
def extract_chunked_audio(audio_array, chunk_length = SEGMENT_LEN, overlap = OVERLAP_LEN):
    chunk_length = chunk_length * SAMPLING_RATE
    overlap = overlap * SAMPLING_RATE
    
    chunks = []
    start = 0
    while start + chunk_length <= len(audio_array):
        chunks.append(
            extractor(
                audio_array[start : start + chunk_length],
                sampling_rate = SAMPLING_RATE,
                max_length = chunk_length,
                truncation = True
            )["input_values"].squeeze(0)
        )
        start += (chunk_length - overlap)
    
    if start < len(audio_array):
        last_chunk = audio_array[start:]
        padded_last_chunk = np.pad(last_chunk, (0, chunk_length - len(last_chunk)))
        chunks.append(
            extractor(
                padded_last_chunk,
                sampling_rate = SAMPLING_RATE,
                max_length = chunk_length,
                truncation = True
            )["input_values"].squeeze(0)
        )
    
    return chunks

In [14]:
def preprocess(row):
    chunked_batched_data = {}
    inputs = extract_chunked_audio(row["audio"]["array"])
    chunked_batched_data["input_values"] = inputs
    chunked_batched_data["common_name"] = [row["common_name"]] * len(inputs)
    return chunked_batched_data

In [15]:
def concate(batch):
   return {
       "concate_input_values": [chunk for chunks in batch["input_values"] for chunk in chunks],
       "chunked_common_name": [label for chunks in batch["common_name"] for label in chunks]
   }

In [16]:
# dataset = dataset.cast_column("audio", Audio(sampling_rate = SAMPLING_RATE))
dataset = dataset.map(
    preprocess,
    remove_columns = "audio",
    batched = False,
    num_proc = 16,
    writer_batch_size = 500
)
dataset = dataset.map(
    concate,
    remove_columns = ["input_values", "common_name"],
    batched = True,
    batch_size = 16,
    num_proc = 16,
    writer_batch_size = 500
)
len(dataset[0]["concate_input_values"])

Map (num_proc=16):   0%|          | 0/5080 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/5080 [00:00<?, ? examples/s]

64

In [17]:
dataset = dataset.rename_column("concate_input_values", "input_values")
dataset = dataset.rename_column("chunked_common_name", "label")
dataset = dataset.shuffle(seed = 42)

In [18]:
sampled_dataset = Dataset.from_dict({"input_values": [], "label": []})

for label in selected_labels:
    temp_dataset = dataset.filter(
        lambda row: row["label"] == label,
        batched = False,
        num_proc = 8,
        writer_batch_size = 50
    )
    temp_dataset = temp_dataset.shuffle(seed = 42).select(range(350))
    sampled_dataset = concatenate_datasets([sampled_dataset, temp_dataset])

del temp_dataset
sampled_dataset = sampled_dataset.shuffle(seed = 42)

Filter (num_proc=8):   0%|          | 0/28284 [00:00<?, ? examples/s]

Filter (num_proc=8):   0%|          | 0/28284 [00:00<?, ? examples/s]

Filter (num_proc=8):   0%|          | 0/28284 [00:00<?, ? examples/s]

Filter (num_proc=8):   0%|          | 0/28284 [00:00<?, ? examples/s]

Filter (num_proc=8):   0%|          | 0/28284 [00:00<?, ? examples/s]

...

Filter (num_proc=8):   0%|          | 0/28284 [00:00<?, ? examples/s]

Filter (num_proc=8):   0%|          | 0/28284 [00:00<?, ? examples/s]

Filter (num_proc=8):   0%|          | 0/28284 [00:00<?, ? examples/s]

Filter (num_proc=8):   0%|          | 0/28284 [00:00<?, ? examples/s]

Filter (num_proc=8):   0%|          | 0/28284 [00:00<?, ? examples/s]

In [19]:
dataset = sampled_dataset.class_encode_column("label")
del sampled_dataset

Flattening the indices:   0%|          | 0/18900 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/18900 [00:00<?, ? examples/s]

In [20]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [21]:
dataset = dataset.train_test_split(test_size = 0.2)

In [22]:
from ced_model.modeling_ced import CedForAudioClassification

model = CedForAudioClassification.from_pretrained(
    "mispeech/ced-base",
    outputdim = len(id2label),
    ignore_mismatched_sizes = True,
    num_labels = len(id2label),
    label2id = label2id,
    id2label = id2label
)

model.freeze_encoder()
model.config.loss = "CrossEntropyLoss"

/users/labnet5/gr5/abahari/Documents/Fridays/BirdCallLabeling/env-labeling/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/24.9k [00:00<?, ?B/s]

class_labels_indices.csv:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

Some weights of CedForAudioClassification were not initialized from the model checkpoint at mispeech/ced-base and are newly initialized because the shapes did not match:
- outputlayer.1.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([54]) in the model instantiated
- outputlayer.1.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([54, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# model.config

In [24]:
accuracy = evaluate.load("accuracy")

In [25]:
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions[0], axis = 1)
    return accuracy.compute(predictions = predictions, references = eval_pred.label_ids)

In [26]:
training_args = TrainingArguments(
    output_dir = "checkpoints-10-2",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-3,               # 3e-4, 5e-4
    per_device_train_batch_size = 32,   # 8
    # gradient_accumulation_steps = 4,
    eval_accumulation_steps = 2,
    per_device_eval_batch_size = 32,    # 8
    num_train_epochs = 10,
    warmup_ratio = 0,
    logging_steps = 10,
    load_best_model_at_end = True,
    metric_for_best_model = "eval_loss",
    dataloader_num_workers = 16
    # fp16 = True
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    tokenizer = extractor,
    compute_metrics = compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,3.351000,3.347308,0.608466
2,3.283300,3.278616,0.652910
3,3.212400,3.246526,0.692063
4,3.195800,3.230722,0.700529
5,3.198300,3.218142,0.722487
6,3.181000,3.211842,0.732011
7,3.184000,3.206649,0.741270
8,3.165200,3.203164,0.748942
9,3.156900,3.201262,0.747619
10,3.169300,3.201169,0.748413


/users/labnet5/gr5/abahari/Documents/Fridays/BirdCallLabeling/env-labeling/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/users/labnet5/gr5/abahari/Documents/Fridays/BirdCallLabeling/env-labeling/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/users/labnet5/gr5/abahari/Documents/Fridays/BirdCallLabeling/env-labeling/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use

TrainOutput(global_step=4730, training_loss=3.2314738574038095, metrics={'train_runtime': 1745.9094, 'train_samples_per_second': 86.602, 'train_steps_per_second': 2.709, 'total_flos': 4.960329810173338e+18, 'train_loss': 3.2314738574038095, 'epoch': 10.0})